<a href="https://colab.research.google.com/github/Nurdaylight/An-Econ-771/blob/main/Speed_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:

# save to    /PS1
import timeit
import urllib.request

url = "https://raw.githubusercontent.com/Nurdaylight/An-Econ-771/main/PS1/data/ps1.csv"
file_path = "/content/ps1.csv"

# Download the file
with urllib.request.urlopen(url) as response:
    content = response.read().decode('utf-8')

with open(file_path, "w") as f:
    f.write(content)

print("Downloaded to", file_path)



with open(file_path, "r") as f:
    next(f)

    data_v = [float(val) for line in f for val in line.strip().split(",")]

#Slice the vector to get each variable

y=data_v[::3]
x1=data_v[1::3]
x2=data_v[2::3]


#Find means
l=len(y)

mean_y=sum(y)/l
mean_x1=sum(x1)/l
mean_x2=sum(x2)/l

#Demean
y=[v - mean_y for v in y]
x1=[v - mean_x1 for v in x1]
x2=[v - mean_x2 for v in x2]

#Create x'y elements
x1y = sum([a*b for a, b in zip(x1, y)])
x2y = sum([a*b for a, b in zip(x2, y)])

#Create elements of inverse of x'x (as not to create matrix for no reason)
x1x1 = sum([a*b for a, b in zip(x1, x1)])
x1x2 = sum([a*b for a, b in zip(x1, x2)])
x2x2 = sum([a*b for a, b in zip(x2, x2)])
#Determinant
inv_det=1/(x1x1*x2x2-x1x2*x1x2)
z=-x1x2*inv_det

beta1=x2x2*inv_det*x1y+z*x2y

beta2=z*x1y+x1x1*inv_det*x2y

beta0=mean_y-mean_x1*beta1-mean_x2*beta2
print("        Intercept: ",beta0)
print("Coefficient on x1: ",beta1)
print("Coefficient on x2: ",beta2)



Downloaded to /content/ps1.csv
        Intercept:  1.8013655409730749
Coefficient on x1:  3.4107893360918777
Coefficient on x2:  0.9895123709648748


In [107]:
from time import perf_counter


In [116]:

#Find means
l=len(y)



times=[]
for _ in range(10000):
  start=perf_counter()
  mean_y=sum(y)/l
  mean_x1=sum(x1)/l
  mean_x2=sum(x2)/l

  #Demean
  y= [v - mean_y for v in y]
  x1=[v - mean_x1 for v in x1]
  x2=[v - mean_x2 for v in x2]
  x1y = x2y = x1x1 = x1x2 = x2x2 = 0.0
  #Create x'y elements
  for a, b, c in zip(x1, x2, y):
      x1y  += a * c
      x2y  += b * c
      x1x1 += a * a
      x1x2 += a * b
      x2x2 += b * b

  inv_det=1/(x1x1*x2x2-x1x2*x1x2)
  z=-x1x2*inv_det

  beta1=x2x2*inv_det*x1y+z*x2y

  beta2=z*x1y+x1x1*inv_det*x2y

  beta0=mean_y-mean_x1*beta1-mean_x2*beta2


  end=perf_counter()
  times.append((end - start) * 1000)  # ms
print("mean (ms):", sum(times) / len(times))
print("min  (ms):", min(times))
print("max  (ms):", max(times))

mean (ms): 0.023557433699579633
min  (ms): 0.02177299984396086
max  (ms): 0.47499599986622343


In [109]:
print(beta0,beta1,beta2)# Flagging beta 0 error

1.0239738436986447e-16 3.410789336091878 0.9895123709648743


In [115]:
times=[]
import numpy as np
X = np.column_stack([np.ones(len(y)), x1, x2])
for _ in range(10000):
  start=perf_counter()
  XtX = X.T @ X
  Xty = X.T @ y
  beta = np.linalg.solve(XtX, Xty)
  end=perf_counter()
  times.append((end - start) * 1000)  # ms
print("mean (ms):", sum(times) / len(times))
print("min  (ms):", min(times))
print("max  (ms):", max(times))

mean (ms): 0.01891174709912775
min  (ms): 0.01611400011825026
max  (ms): 0.6301869998424081


In [111]:
beta1

3.410789336091878